### Análisis de Casos COVID y Dependencias con Vacunas
La práctica final de nuestro grupo constará del análisis del dataset publicado por el European Centre for Disease Prevention and Control sobre casos acumulativos de COVID-19 en 2020 y sus muertes asociadas, como parte del paper Demographic and public health characteristics explain large part of variability in COVID-19 mortality across countries.

Link al paper:
https://academic.oup.com/eurpub/article/31/1/12/6106197

### Análisis Exploratorio

Para comenzar con la práctica, comenzaremos por importar el dataset y aplicar diferentes técnicas explorativas vistas para comprender mejor con qué información contamos, y la relación entre las diferentes características contempladas.

Luego, decidiremos cuál de las preguntas propuestas elegiremos para basar el resto de la práctica integradora.

Estas preguntas pueden ser:

 - ¿Existe alguna correlación entre la obligatoriedad de la vacuna BCG y los
casos?
 - ¿Se pueden encontrar valores de interés a nuestro análisis? ¿Cómo se
comportan estos datos?


In [11]:
# Importamos librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={
    'axes.facecolor': 'black',
    'figure.facecolor': 'black',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'axes.titlecolor': 'white'
})

# Cargamos el dataset
data = pd.read_csv("./data/covid-dataset.txt")

# Cambiamos el nombre de la última columna para que sea un poco más corto
data.rename(columns={'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000': 'deathsPerMillCasesIn14Days'},inplace=True)

# Visualizamos las primeras filas del dataset
data.head(1)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,casesPerMillIn14Days
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779


### Análisis y clasificación de variables

Con un simple vistazo y sin contemplar la calidad y consistencia de las características del dataset, podemos realizar una pequeña descripción y clasificación de las variables que presenta:

| Variable               | Tipo | Descripción |
| ---------------------- | -----| ------------|
| dateRep                |      |             |
| day                    |      |             |
| month                  |      |             |
| year                   |      |             |
| cases                  |      |             |
| deaths                 |      |             |
| countriesAndTerritories|      |             |
| geoId      |           |      |             |
| countryterritoryCode   |      |             |
| popData2019            |      |             |
| continentExp           |      |             |
| casesPerMillIn14Days   |      |             |